In [52]:
import pandas as pd
import numpy as np
import seaborn as sns

C:\Users\ACER\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [53]:
test = pd.read_csv('../data/test.csv', dtype={'ID': 'int32', 'shop_id': 'int32', 
                                                  'item_id': 'int32'})
item_categories = pd.read_csv('../data/item_categories.csv', 
                              dtype={'item_category_name': 'str', 'item_category_id': 'int32'})
items = pd.read_csv('../data/items.csv', dtype={'item_name': 'str', 'item_id': 'int32', 
                                                 'item_category_id': 'int32'})
shops = pd.read_csv('../data/shops.csv', dtype={'shop_name': 'str', 'shop_id': 'int32'})
sales = pd.read_csv('../data/sales_train.csv', parse_dates=['date'], 
                    dtype={'date': 'str', 'date_block_num': 'int32', 'shop_id': 'int32', 
                          'item_id': 'int32', 'item_price': 'float32', 'item_cnt_day': 'int32'})

In [54]:
# train = sales.join(items, on='item_id', rsuffix='_').join(shops, on='shop_id', rsuffix='_').join(item_categories, on='item_category_id', rsuffix='_').drop(['item_id_', 'shop_id_', 'item_category_id_'], axis=1)

In [55]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,2013-02-01,0,59,22154,999.000000,1
1,2013-03-01,0,25,2552,899.000000,1
2,2013-05-01,0,25,2552,899.000000,-1
3,2013-06-01,0,25,2554,1709.050049,1
4,2013-01-15,0,25,2555,1099.000000,1


In [56]:
train['date']

0         2013-02-01
1         2013-03-01
2         2013-05-01
3         2013-06-01
4         2013-01-15
             ...    
2935844   2015-10-10
2935845   2015-09-10
2935846   2015-10-14
2935847   2015-10-22
2935848   2015-03-10
Name: date, Length: 2935849, dtype: datetime64[ns]

In [58]:
train['date'].max().date()

datetime.date(2015, 12, 10)

In [60]:
train.query('date == "2015-12-10"').head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,shop_name,item_category_name
2882338,2015-12-10,33,45,13881,659.0,1,ЛЕПС ГРИГОРИЙ The Best 3CD (фирм.),55,"Самара ТЦ ""ПаркХаус""",Музыка - CD локального производства
2882341,2015-12-10,33,45,14931,799.0,1,Магический шар 8 оригинальный,69,"Самара ТЦ ""ПаркХаус""",Подарки - Сувениры
2882381,2015-12-10,33,45,15257,399.0,1,Мягкая игрушка Angry Birds Черная птица 30см,63,"Самара ТЦ ""ПаркХаус""",Подарки - Мягкие игрушки
2882394,2015-12-10,33,45,15395,699.0,1,Мягкая игрушка Мадагаскар Шкипер,63,"Самара ТЦ ""ПаркХаус""",Подарки - Мягкие игрушки
2882424,2015-12-10,33,45,15255,399.0,1,Мягкая игрушка Angry Birds Зеленая свинка 30см,63,"Самара ТЦ ""ПаркХаус""",Подарки - Мягкие игрушки


In [6]:
train.describe()

date_block_num       shop_id       item_id    item_price  item_cnt_day  \
count    2.935849e+06  2.935849e+06  2.935849e+06  2.935849e+06  2.935849e+06   
mean     1.456991e+01  3.300173e+01  1.019723e+04  8.906234e+02  1.242641e+00   
std      9.422988e+00  1.622697e+01  6.324297e+03  1.726440e+03  2.618834e+00   
min      0.000000e+00  0.000000e+00  0.000000e+00 -1.000000e+00 -2.200000e+01   
25%      7.000000e+00  2.200000e+01  4.476000e+03  2.490000e+02  1.000000e+00   
50%      1.400000e+01  3.100000e+01  9.343000e+03  3.990000e+02  1.000000e+00   
75%      2.300000e+01  4.700000e+01  1.568400e+04  9.990000e+02  1.000000e+00   
max      3.300000e+01  5.900000e+01  2.216900e+04  3.079800e+05  2.169000e+03   

       item_category_id  
count      2.935849e+06  
mean       4.000138e+01  
std        1.710076e+01  
min        0.000000e+00  
25%        2.800000e+01  
50%        4.000000e+01  
75%        5.500000e+01  
max        8.300000e+01

In [7]:
train[train['item_price'] < 0]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,shop_name,item_category_name
484683,2013-05-15,4,32,2973,-1.0,1,"DmC Devil May Cry [PS3, русские субтитры]",19,"Москва ТЦ ""Серебряный Дом""",Игры - PS3


In [18]:
train['date'].max().date().month

12

In [12]:
print('Min date from train set: %s' % train['date'].min().date())
print('Max date from train set: %s' % train['date'].max().date())

Min date from train set: 2013-01-01
Max date from train set: 2015-12-10


In [19]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [20]:
train['date_block_num'].nunique()

34

In [21]:
sales['date'] = pd.to_datetime(sales['date'], format="%Y %m %d")

In [22]:
lk_train = train[train['shop_id'].isin(test_shop_ids)]

lk_train = lk_train[lk_train['item_id'].isin(test_item_ids)]

In [ ]:
np.all(lk_train['item_id'].isin(test_item_ids).values)

In [23]:
print('Data set size before leaking:', train.shape[0])
print('Data set size after leaking:', lk_train.shape[0])

Data set size before leaking: 2935849
Data set size after leaking: 1224439


In [24]:
lk_train['date_block_num'].nunique()

34

In [25]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 10 columns):
 #   Column              Dtype         
---  ------              -----         
 0   date                datetime64[ns]
 1   date_block_num      int32         
 2   shop_id             int32         
 3   item_id             int32         
 4   item_price          float32       
 5   item_cnt_day        int32         
 6   item_name           object        
 7   item_category_id    int32         
 8   shop_name           object        
 9   item_category_name  object        
dtypes: datetime64[ns](1), float32(1), int32(5), object(3)
memory usage: 156.8+ MB


In [31]:
lk_train['date'].max().date()

datetime.date(2015, 12, 10)

In [32]:
sales['year'] = sales['date'].dt.year

In [33]:
train_monthly.tail()

date  date_block_num  shop_id  item_category_id  item_id  \
2935844 2015-10-10              33       25                55     7409   
2935845 2015-09-10              33       25                55     7460   
2935846 2015-10-14              33       25                55     7459   
2935847 2015-10-22              33       25                57     7440   
2935848 2015-03-10              33       25                55     7460   

         item_price  item_cnt_day  
2935844       299.0             1  
2935845       299.0             1  
2935846       349.0             1  
2935847       299.0             1  
2935848       299.0             1

In [34]:
train_monthly['date'].max().date()

datetime.date(2015, 12, 10)

In [ ]:
l = [[1, 2, 3], [1, None, 4], [2, 1, 3], [1, 2, 2]]
df = pd.DataFrame(l, columns=["a", "b", "c"])

In [ ]:
df

In [ ]:
df.groupby(by=['a', "b"]).sum()

In [35]:
train_monthly['date_block_num'].nunique()

34

In [36]:
train_monthly = train_monthly.sort_values('date').groupby(['date_block_num', 'shop_id', 'item_category_id', 'item_id'], as_index=False).agg({'item_price':['sum', 'mean'], 'item_cnt_day':['sum', 'mean','count']})
train_monthly.columns = ['date_block_num', 'shop_id', 'item_category_id', 'item_id', 'item_price', 'mean_item_price', 'item_cnt', 'mean_item_cnt', 'transactions']

In [37]:
temp.head()

,date_block_num,shop_id,item_id,item_price,month,year,item_cnt_month
0,0,0,32,221.0,1,2013,4
1,0,0,32,221.0,3,2013,2
2,0,0,33,347.0,1,2013,2
3,0,0,33,347.0,3,2013,1
4,0,0,35,247.0,1,2013,1


In [38]:
train.head()

date_block_num      0
shop_id             0
item_category_id    0
item_id             0
item_price          0
mean_item_price     0
item_cnt            0
mean_item_cnt       0
transactions        0
dtype: int64

In [39]:
train = pd.merge(train, items, how='left', on=['item_id'])
train = pd.merge(train, item_categories, how='left', on=['item_category_id'])
train = pd.merge(train, shops, how='left', on=['shop_id'])
train.head()

,date_block_num,shop_id,item_id,item_price,month,year,item_cnt_month,item_cnt_mean,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name
0,0,0,32,221.0,1,2013,4,2.666667,0.0,1+1,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
1,0,0,32,221.0,3,2013,2,2.666667,0.0,1+1,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"
2,0,0,33,347.0,1,2013,2,1.500000,0.0,1+1 (BD),37,Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран"
3,0,0,33,347.0,3,2013,1,1.500000,0.0,1+1 (BD),37,Кино - Blu-Ray,"!Якутск Орджоникидзе, 56 фран"
4,0,0,35,247.0,1,2013,1,1.875000,0.0,10 ЛЕТ СПУСТЯ,40,Кино - DVD,"!Якутск Орджоникидзе, 56 фран"


In [45]:
test['month'] = 11
test['year'] = 2015
test['date_block_num'] = 34

In [40]:
test.head()

,ID,shop_id,item_id,month,year,date_block_num
0,0,5,5037,11,2015,34
1,1,5,5320,11,2015,34
2,2,5,5233,11,2015,34
3,3,5,5232,11,2015,34
4,4,5,5268,11,2015,34


In [47]:
test = pd.merge(test, shop_item_mean, how='left', on=['shop_id', 'item_id']).fillna(0)

In [41]:
test.head()

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_mean
0,0,5,5037,11,2015,34,1.083333
1,1,5,5320,11,2015,34,0.000000
2,2,5,5233,11,2015,34,1.250000
3,3,5,5232,11,2015,34,1.000000
4,4,5,5268,11,2015,34,0.000000


In [49]:
test = pd.merge(test, shop_pre_month, how='left', on=['shop_id', 'item_id']).fillna(0)

In [50]:
test = pd.merge(test, items, how='left', on=['item_id']).fillna(0)

In [51]:
test = pd.merge(test, item_categories, how='left', on=['item_category_id']).fillna(0)

In [42]:
test = pd.merge(test, shops, how='left', on=['shop_id']).fillna(0)

In [43]:
test['item_cnt_month'] = 0

In [44]:
test.head()

,ID,shop_id,item_id,month,year,date_block_num,item_cnt_mean,item_cnt_prev_month,item_name,item_category_id,item_category_name,shop_name,item_cnt_month
0,0,5,5037,11,2015,34,1.083333,0.0,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0
1,1,5,5320,11,2015,34,0.000000,0.0,ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,"Вологда ТРЦ ""Мармелад""",0
2,2,5,5233,11,2015,34,1.250000,1.0,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,"Вологда ТРЦ ""Мармелад""",0
3,3,5,5232,11,2015,34,1.000000,0.0,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,"Вологда ТРЦ ""Мармелад""",0
4,4,5,5268,11,2015,34,0.000000,0.0,"Need for Speed [PS4, русская версия]",20,Игры - PS4,"Вологда ТРЦ ""Мармелад""",0


In [45]:
from sklearn.preprocessing import LabelEncoder

In [47]:
train_monthly['date_block_num'].nunique()

34

In [51]:
train_monthly['date'].max().date()

KeyError: 'date'

In [50]:
train_monthly.query('year == 2015 and month == 10')

,date_block_num,shop_id,item_id,item_category_id,item_price,mean_item_price,item_cnt,mean_item_cnt,transactions,year,month
